In [1]:
# Install required packages
!pip install sentencepiece
!pip install transformers
!pip install transformers accelerate
!pip install torch==1.11.00.676617


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement torch==1.11.00.676617 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0)
ERROR: No matching distribution found for torch==1.11.00.676617

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd

In [3]:
# Load the datasets
df_chungli_ao = pd.read_csv("data/Chungli_Ao_train_set.csv")
df_english = pd.read_csv("data/English_dataset.csv")
df_german = pd.read_csv("data/german_sentiment.csv", sep=";")
df_mizo = pd.read_csv("data/Mizo_Training_Software_project.csv")
df_russian = pd.read_csv("data/Russian_fixed.csv")
df_telugu = pd.read_csv("data/Telugu_soft_ware_project.csv")

In [4]:
'''
We working on Chungli AO

1 step: Data Collection.
2 step: Code  (Traditional ML vs pretrained models (XLMR,MBERT)
3 step: Multilingual data augmentation.English,German,Telugu, Russian and  Mizo is used in the training set.
'''

'\nWe working on Chungli AO\n\n1 step: Data Collection.\n2 step: Code  (Traditional ML vs pretrained models (XLMR,MBERT)\n3 step: Multilingual data augmentation.English,German,Telugu, Russian and  Mizo is used in the training set.\n'

In [5]:
# Concatenate the datasets
df_chungli_english = pd.concat([df_chungli_ao, df_english], ignore_index=True)
df_chungli_german = pd.concat([df_chungli_ao, df_german], ignore_index=True)
df_chungli_mizo = pd.concat([df_chungli_ao, df_mizo], ignore_index=True)
df_chungli_russian = pd.concat([df_chungli_ao, df_russian], ignore_index=True)
df_chungli_russian = df_chungli_russian[['Text', 'Sentiment']]
df_chungli_telugu = pd.concat([df_chungli_ao, df_telugu], ignore_index=True)

In [ ]:
# change this variable to change the data you're working with
df = df_chungli_mizo

In [7]:
# Converting the labels to numbers
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

df['Sentiment'] = le.fit_transform(df.Sentiment.values)

In [8]:
# Shuffle the dataframe
from sklearn.utils import shuffle
df = shuffle(df)

In [ ]:
df.head()

,Text,Sentiment
5051,"Pu TBC-A sawi tha rem rem hle mai, ZPM go go",1
4977,"Ava sawidik chiang em,kan sawrkar thartur chu ...",1
74,laptop nung agüja aliba 47.4Whr mapang iba fil...,1
4573,He mess aiin student film production value tha...,0
3217,lir. Tarutsü nung iba sentong takok nguyonga a...,0
...,...,...
8244,Political hruaitute hian strategic misinformat...,0
2138,"""Ajisüaka, osang tajung kaji meshiranger, anun...",1
7098,He hlaa harmonies te hian thlamuanna min pe a....,1
2902,achiajem mazüoktetba yimya tamajung melenshits...,0


In [10]:
# Count the number of true and fake labels
True_labels = (df['Sentiment'] == 1).sum()
fake_labels = (df['Sentiment'] == 0).sum()
print(f"Number of True labels: {True_labels}")
print(f"Number of Fake labels: {fake_labels}")

Number of True labels: 5158
Number of Fake labels: 5161


In [11]:
# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels_str, val_labels_str = train_test_split(list(df['Text']), list(df['Sentiment']), test_size=.2)

In [12]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Blue7Bird/chungli_ao_bert_model", model_max_length=512)
model = AutoModelForSequenceClassification.from_pretrained("Blue7Bird/chungli_ao_bert_model")

c:\Users\lenao\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
# Tokenize the texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [14]:
import torch

In [15]:
# Define custom dataset class
class spanish_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [16]:
# Encode labels
train_labels = le.fit_transform(train_labels_str)
val_labels = le.transform(val_labels_str)

# Create dataset objects
train_dataset = spanish_Dataset(train_encodings, train_labels)
val_dataset = spanish_Dataset(val_encodings, val_labels)

In [17]:
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer

class CustomClassifier(nn.Module):
    def __init__(self, model_name, num_labels):
        super(CustomClassifier, self).__init__()
        self.base_model = AutoModel.from_pretrained(model_name)
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(self.base_model.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.base_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.last_hidden_state[:, 0, :]  # Use the representation of [CLS] token
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.classifier.out_features), labels.view(-1))

        return (loss, logits) if loss is not None else logits

In [18]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import torch

In [19]:
model_name = "Blue7Bird/chungli_ao_bert_model"
num_labels = 2  # Adjust this based on your specific task
custom_model = CustomClassifier(model_name, num_labels)

In [20]:
# Define metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='micro')
    acc = accuracy_score(labels, preds)
    return {'Accuracy': acc, 'F1': f1, 'Precision': precision, 'Recall': recall}

In [21]:
#train_dataset.head()

In [22]:
# Define training arguments
from transformers import Trainer, TrainingArguments, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=15,  # batch size per device during training
    per_device_eval_batch_size=15,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
    do_eval=True,
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    eval_steps=10
)

# Initialize the Trainer
trainer = Trainer(
    model=custom_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics,     # compute metrics function
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# Train the model
trainer.train()

# Evaluate the model
metrics = trainer.evaluate()
print(metrics)

c:\Users\lenao\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  2%|▏         | 10/551 [11:10<10:25:46, 69.40s/it]

{'loss': 0.6986, 'grad_norm': 9.783676147460938, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.02}


                                                   
  2%|▏         | 10/551 [1:26:20<10:25:46, 69.40s/it]

{'eval_loss': 0.7084096074104309, 'eval_Accuracy': 0.500484496124031, 'eval_F1': 0.500484496124031, 'eval_Precision': 0.500484496124031, 'eval_Recall': 0.500484496124031, 'eval_runtime': 4510.0013, 'eval_samples_per_second': 0.458, 'eval_steps_per_second': 0.031, 'epoch': 0.02}


  4%|▎         | 20/551 [1:37:27<17:48:16, 120.71s/it]  

{'loss': 0.7115, 'grad_norm': 10.184102058410645, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.04}


In [ ]:
test_df = pd.read_csv("data/Test_data_Chungli_ao.csv")

In [ ]:
len(test_df)

In [ ]:
test_df.head()

In [ ]:
test_df.tail()

In [ ]:
test_texts = list(test_df['Text'])

In [ ]:
test_df['Sentiment'] = le.fit_transform(test_df.Sentiment.values)

In [ ]:
test_labels = test_df['Sentiment']

In [ ]:
len(test_labels)

In [ ]:
test_encodings = tokenizer(test_texts, truncation=True, padding=True)
test_dataset = spanish_Dataset(test_encodings, test_labels)

In [ ]:
test_metrics = trainer.evaluate(test_dataset)

In [ ]:
print(test_metrics)